# 07: Calculate sensitivities

In this script, we calculate the sensitivities with respect to monetization and location.

In [1]:
%run common_definitions.py

In [2]:
import pandas as pd
import xarray as xr
import numpy as np

from premise.geomap import Geomap

In [3]:
output_fp = "../output/" + BW_PROJECTNAME

### Load the impact file and the monetization factors sample

In [4]:
combined_impacts = pd.read_csv(output_fp+"/impacts_combined.csv").set_index(
    ["short name", "sector", "scenario", "year", "location"]
)

In [5]:
mfs_mc_sample_cc = xr.open_dataarray("../data/mfs_monte_carlo_sample_extended_euro{}.nc".format(str(EURO_REF_YEAR)))

### Define a regional filter function

In [6]:
def regional_filter(df, iam_regions):
    geo = Geomap(model="remind")

    filter_list = []
    for region in iam_regions:
        filter_list += geo.iam_to_ecoinvent_location(region)

    return df[df["location"].isin(filter_list)]

### Sensitivity calculation

In [7]:
def calculate_sensitivities(df, region, total_effect=True):
    all_sectors = list(df.index.unique(level="sector"))

    new_index = []
    values = []
    counts = []
    for sector in all_sectors:
        for scen in SCENARIOS:
            for year in YEARS:
                print("Calculating sensitivities for {}, {}, {}".format(sector, scen, year))

                sel = df.loc[pd.IndexSlice[:, sector, scen, year, :]]
                techs = list(sel.index.unique(level="short name"))
                for tech in techs:
                    new_index.append((tech, sector, scen, year, region))
                    sel2 = sel.loc[pd.IndexSlice[tech, :]]

                    # how many locations?
                    counts.append(len(sel2))

                    # calculate costs
                    costs = xr.DataArray(
                        np.tensordot(sel2[list(mfs_mc_sample_cc.coords["LCIA method"].values)].to_numpy(),
                                    mfs_mc_sample_cc.to_numpy(),axes=([1,], [2,])),
                        coords = {
                            "location": list(sel2.index),
                            "impact category": list(mfs_mc_sample_cc.coords["impact category"].values),
                            "sample index": list(range(mfs_mc_sample_cc.shape[1]))
                        }
                    )
                    total = costs.sum(dim="impact category").expand_dims(dim={"impact category": ["total"]}, axis=1)
                    costs = xr.concat((costs, total), dim="impact category")

                    # full variance
                    full_var = costs.var(dim=["location", "sample index"])

                    # sensitivity indices
                    s_loc = costs.mean(dim="sample index").var(dim="location") / full_var
                    s_mvc = costs.mean(dim="location").var(dim="sample index") / full_var

                    if total_effect:
                        s_loc_te = 1 - s_mvc
                        s_mvc_te = 1 - s_loc

                        s_loc_te = s_loc_te.sortby("impact category")
                        s_mvc_te = s_mvc_te.sortby("impact category")

                        values.append(list(s_loc_te.to_numpy())+list(s_mvc_te.to_numpy()))
                    else:
                        # collect values, making sure that sorting is correct
                        s_loc = s_loc.sortby("impact category")
                        s_mvc = s_mvc.sortby("impact category")
                        values.append(list(s_loc.to_numpy())+list(s_mvc.to_numpy()))

    # collect columns from last calculation
    if total_effect:
        cols = list(s_loc_te.coords["impact category"].values)
        new_col_index = [("location", ic) for ic in cols] + [("monetization", ic) for ic in cols]
    else:
        cols = list(s_loc.coords["impact category"].values)
        new_col_index = [("location", ic) for ic in cols] + [("monetization", ic) for ic in cols]

    df_s = pd.DataFrame(
        np.array(values),
        index=pd.MultiIndex.from_tuples(new_index, names=["short name", "sector", "scenario", "year", "region"]),
        columns=pd.MultiIndex.from_tuples(new_col_index, names=["sensitivity variable", "impact category"])
    )

    s_counts = pd.Series(
        counts,
        index=pd.MultiIndex.from_tuples(new_index, names=["short name", "sector", "scenario", "year", "region"])
    )

    return df_s.reset_index().melt(
        id_vars=["short name", "sector", "scenario", "year", "region"], ignore_index=False).pivot(
            index=["short name", "sector", "scenario", "year", "region", "sensitivity variable"],
            columns="impact category",
            values="value"
            ), s_counts

In [8]:
dflist = []
slist = []
sensitivities, counts = calculate_sensitivities(combined_impacts, "all")
dflist.append(sensitivities)
slist.append(counts)

for region in REMIND_REGIONS:
    df = regional_filter(combined_impacts.reset_index(), [region,]).set_index(
        ["short name", "sector", "scenario", "year", "location"]
    )
    sensitivities, counts = calculate_sensitivities(df, region)
    dflist.append(sensitivities)
    slist.append(counts)

Calculating sensitivities for electricity production, SSP2-NPi, 2020
Calculating sensitivities for electricity production, SSP2-NPi, 2030
Calculating sensitivities for electricity production, SSP2-NPi, 2040
Calculating sensitivities for electricity production, SSP2-NPi, 2050
Calculating sensitivities for electricity production, SSP2-PkBudg500, 2020
Calculating sensitivities for electricity production, SSP2-PkBudg500, 2030
Calculating sensitivities for electricity production, SSP2-PkBudg500, 2040
Calculating sensitivities for electricity production, SSP2-PkBudg500, 2050
Calculating sensitivities for residential heating, SSP2-NPi, 2020
Calculating sensitivities for residential heating, SSP2-NPi, 2030
Calculating sensitivities for residential heating, SSP2-NPi, 2040
Calculating sensitivities for residential heating, SSP2-NPi, 2050
Calculating sensitivities for residential heating, SSP2-PkBudg500, 2020
Calculating sensitivities for residential heating, SSP2-PkBudg500, 2030
Calculating sens

In [9]:
pd.concat(dflist, axis=0).to_csv(output_fp+"/sensitivities_total_effect.csv")
pd.concat(slist, axis=0).to_csv(output_fp+"/counts_locations.csv")